Names, Matriculation Numbers:  
Jie Cheng: 5527191  
Markus Flicke: 5489277  
Bin Wei: 4277156

### Cleaning

In [5]:
import pandas as pd
from fancyimpute import *

In [6]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
target = df_train.target
df_train = df_train.drop('target', axis = 1)

Some ridicolous dtype conversions as NaN is not allowed for integers

In [7]:
df_train.age = df_train.age.astype(float)
df_test.marriage = df_test.marriage.astype(float)
df_test.PAY_6 = df_test.PAY_6.astype(float)

I wanted both dataframes together, so that the one-hot-encoding is consistent. Some categorical values only exist in train

In [8]:
df_both = pd.concat([df_train, df_test], ignore_index=True)

### Data impurity
For some stupid reason, the data contains value 33. I am assuming it means 3

In [9]:
df_both.marriage = df_both.marriage.apply(lambda x: 3 if x == 33 else x)

### Categorical variable -> One-hot-encoding

In [10]:
df = pd.get_dummies(df_both, columns = ['education', 'sex', 'marriage', 'profession'])

In [11]:
for i in range(1,6):
    df[f'PAY_{i}_minus2'] = df[f'PAY_{i}'] == -2
    df[f'PAY_{i}_minus1'] = df[f'PAY_{i}'] == -1

### Min Max normalisation

In [12]:
# normalisation_columns = ['limit_balance', 'n_children', 'age'] + \
# [f'PAY_{i}' for i in range(1,6)] +\
# [f'BILL_AMT{i}' for i in range(1,6)] +\
# [f'PAY_AMT{i}' for i in range(1,6)]

In [13]:
# for col_name in normalisation_columns:
#     col = df[col_name]
#     df[col_name] = (col-col.min())/(col.max()-col.min())

### KNN Imputation with [Fancyimpute](https://stackoverflow.com/questions/45321406/missing-value-imputation-in-python-using-knn)

In [14]:
%%time
X_imputed = KNN(k=3, verbose = False).solve(df.values, df.isnull().values)

Wall time: 6min 9s


In [24]:
imputed_df = pd.DataFrame(X_imputed, columns = df.columns)

In [25]:
df_train_imputed = imputed_df.iloc[:15000]
df_train_imputed['target'] = target
df_test_imputed = imputed_df.iloc[15000:]

c:\users\m\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
df_train_imputed.to_csv('train_imputed.csv', index = False)
df_test_imputed.to_csv('test_imputed.csv', index = False)